In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress
from sklearn.ensemble import RandomForestClassifier

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, concatenate, Conv2D
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

In [3]:
def load_5models_from_disk():
    models = []
    for i in range(5):
        json_file = open("./model" + str(i) +".json", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        # load weights into new model
        loaded_model.load_weights("temp" + str(i) +".hdf5")
        print("Loaded model from disk")

        # evaluate loaded model on test data
        loaded_model.compile(optimizer='rmsprop',
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])
        models.append(loaded_model)
    return models

In [4]:
def shrink_array(array,size):
    
    ratio = float(len(array)) / float(size+1)
    res = []
    for i in range(size):
        res.append(np.mean(array[math.floor(i*ratio):math.ceil((i+2.0)*ratio)], axis = 0))
    return np.array(res)

In [ ]:
gesture_features = pd.DataFrame.from_csv("../../data/gesture_feature_df.csv")
test_data = pd.DataFrame.from_csv("../../data/database/test_data.csv")
gesture_features = gesture_features[(gesture_features.TagName == 'Tag0') | (gesture_features.TagName == 'Tag6')]

groups = test_data.groupby(['TagName','tester_id'])
keys = groups.groups.keys()
test = []
train = []

for i in range(len(gesture_features)):
    if (gesture_features.iloc[i]['TagName'],gesture_features.iloc[i]['tester_id']) in keys:
#     t = gesture_features.loc[(gesture_features.TagName == k[0]) & (gesture_features.tester_id == k[1])]
#     if (len(t) > 0):
        test.append(gesture_features.iloc[i].values)
    else:
        train.append(gesture_features.iloc[i].values)
test = pd.DataFrame(test,columns=gesture_features.columns)
train = pd.DataFrame(train,columns=gesture_features.columns)

test.to_csv("../../data/database/test_gesture_features.csv")
train.to_csv("../../data/database/train_gesture_features.csv")

In [5]:
test = pd.DataFrame.from_csv("../../data/database/test_gesture_features.csv")
train = pd.DataFrame.from_csv("../../data/database/train_gesture_features.csv")

In [6]:
clf = RandomForestClassifier(n_estimators=30)
clf.fit(train.d_change.values.reshape(len(train),1),train.TagName.values)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [7]:
res = clf.predict(test.d_change.values.reshape(len(test),1))

In [8]:
print(classification_report(test.TagName, res))
print(confusion_matrix(test.TagName, res))  

             precision    recall  f1-score   support

       Tag0       0.91      0.95      0.93        21
       Tag6       0.95      0.90      0.93        21

avg / total       0.93      0.93      0.93        42

[[20  1]
 [ 2 19]]


In [9]:
from sklearn.externals import joblib
joblib.dump(clf, './binary_model.pkl') 

['./binary_model.pkl']